## Configurar Workspace

In [ ]:
from azureml.core import Workspace
ws = Workspace.from_config()

print("Azure ML Workspace")
print(f'Name: {ws.name}')

## Crear Experimento

In [ ]:
from azureml.core import Experiment

script_folder = './concrete'
os.makedirs(script_folder, exist_ok=True)

exp = Experiment(workspace=ws, name='concrete')

## Obtener Datos

In [ ]:
from azureml.core import Dataset

dataset_name = 'Concrete Data'
dataset = Dataset.get_by_name(workspace=ws, name=dataset_name)

## Obtener Computo de Entrenamiento

In [ ]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

# choose a name for your cluster
cluster_name = "gpu-cluster"

try:
    compute_target = ComputeTarget(workspace=ws, name=cluster_name)
    print('Found existing compute target')
except ComputeTargetException:
    print('Creating a new compute target...')
    compute_config = AmlCompute.provisioning_configuration(vm_size='STANDARD_NC6', max_nodes=1)

    # create the cluster
    compute_target = ComputeTarget.create(ws, cluster_name, compute_config)

    # can poll for a minimum number of nodes and for a specific timeout. 
    # if no min node count is provided it uses the scale settings for the cluster
    compute_target.wait_for_completion(show_output=True, min_node_count=None, timeout_in_minutes=20)

# use get_status() to get a detailed status for the current cluster. 
print(compute_target.get_status().serialize())

## Crear un Experimento

In [ ]:
from azureml.train.dnn import TensorFlow

est = TensorFlow(source_directory=script_folder,
                 compute_target=compute_target, 
                 entry_script='keras-train.py',
                 framework_version='2.0', 
                 use_gpu=True,
                 pip_packages=['keras<=2.3.1','Pillow','matplotlib'])

In [ ]:
from azureml.pipeline.steps import EstimatorStep

est_step = EstimatorStep(name='train step',
                         estimator=est,
                         estimator_entry_script_arguments=[],
                         inputs=[dataset.as_named_input('concretedata').as_mount()],
                         compute_target=compute_target)

### Ejecutar Experimento

In [ ]:
from azureml.pipeline.core import Pipeline

pipeline = Pipeline(ws, steps=[est_step])
run = exp.submit(pipeline)

In [ ]:
run.wait_for_completion(show_output=True)

## Registrar Modelo

In [ ]:

model = run.find_step_run('train step')[0].register_model(model_name = 'concrete-keras-pipeline', 
                                                            model_path = 'outputs/model/', 
                                                            datasets =[('train test data',dataset)])                           
print(model.name, model.id, model.version, sep='\t')

## Publicar Modelo

In [ ]:
%%writefile score.py
import json
import numpy as np
import os
from keras.models import model_from_json

from azureml.core.model import Model

def init():
    global model
    
    model_root = Model.get_model_path('concrete-keras-pipeline')
    # load json and create model
    json_file = open(os.path.join(model_root, 'model.json'), 'r')
    model_json = json_file.read()
    json_file.close()
    model = model_from_json(model_json)
    # load weights into new model
    model.load_weights(os.path.join(model_root, "model.h5"))   
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    
def run(raw_data):
    data = np.array(json.loads(raw_data)['data'])
    # make prediction
    y_hat = np.argmax(model.predict(data), axis=1)
    return y_hat.tolist()

In [ ]:
from azureml.core.conda_dependencies import CondaDependencies

cd = CondaDependencies.create()
cd.add_tensorflow_conda_package()
cd.add_conda_package('keras<=2.3.1')
cd.add_pip_package("azureml-defaults")
cd.save_to_file(base_directory='./', conda_file_path='myenv.yml')

print(cd.serialize_to_string())

In [ ]:
from azureml.core.webservice import AciWebservice
from azureml.core.model import InferenceConfig
from azureml.core.model import Model
from azureml.core.environment import Environment


myenv = Environment.from_conda_specification(name="myenv", file_path="myenv.yml")
inference_config = InferenceConfig(entry_script="score.py", environment=myenv)

aciconfig = AciWebservice.deploy_configuration(cpu_cores=1,
                                               auth_enabled=True, # this flag generates API keys to secure access
                                               memory_gb=1,
                                               tags={'name': 'concrete', 'framework': 'Keras'},
                                               description='Keras on Concrete with Pipeline')

service = Model.deploy(workspace=ws, 
                           name='keras-concrete-pipeline-svc', 
                           models=[model], 
                           inference_config=inference_config, 
                           deployment_config=aciconfig)

service.wait_for_deployment(True)
print(service.state)

## Test Web Service

In [ ]:
print(service.scoring_uri)

In [ ]:
key1, Key2 = service.get_keys()

In [ ]:
import requests

def test_endpoint(test_img, y):
    input_data = "{\"data\": [" + str(np.array(test_img).tolist()) + "]}"

    headers = {'Content-Type':'application/json', 'Authorization': 'Bearer ' + key1}

    resp = requests.post(service.scoring_uri, input_data, headers=headers)

    print("POST to url", service.scoring_uri)
    print("label:", y)
    print("prediction:", resp.text)

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow import keras

negative_test_img = tf.keras.preprocessing.image.load_img(
    'sample/negative/19751.jpg', grayscale=False, color_mode="rgb", target_size=None, interpolation="nearest"
)

positive_test_img = tf.keras.preprocessing.image.load_img(
    'sample/positive/19754.jpg', grayscale=False, color_mode="rgb", target_size=None, interpolation="nearest"
)

test_endpoint(negative_test_img, 0)
test_endpoint(positive_test_img, 1)

In [ ]:
service.delete()